In [ ]:
# using the pyoscar API

from pyoscar import OSCARClient

client = OSCARClient()

# get all Canadian stations
stations = client.get_stations(country='CAN')

# get all Canadian stations
stations = client.get_stations(program='GAW')

# get invididual station report
stn_leo = client.get_station_report('LEO')


# upload WMDR XML

## instantiate client to OSCAR DEPL (default)
client = OSCARClient(api_token='foo')

## ...or to OSCAR production
client = OSCARClient(api_token='foo', env='prod')

with open('some-wmdr-file.xml') as fh:
    data = fh.read()

response = client.upload(data)

